If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
!pip install datasets transformers

     |████████████████████████████████| 312 kB 7.7 MB/s 
     |████████████████████████████████| 3.8 MB 32.3 MB/s 
     |████████████████████████████████| 134 kB 42.7 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
     |████████████████████████████████| 212 kB 47.3 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 127 kB 42.6 MB/s 
     |████████████████████████████████| 6.5 MB 45.4 MB/s 
     |████████████████████████████████| 895 kB 38.5 MB/s 
     |████████████████████████████████| 596 kB 39.4 MB/s 
     |████████████████████████████████| 144 kB 49.5 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 271 kB 49.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,088 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155332 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.17.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](images/question_answering.png)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict, concatenate_datasets
import pandas as pd

In [ ]:
dataset_df = pd.read_json('qa-database-labeled.json')
dataset_df.head()

,context,question,answers
0,we live in a world where physical exhaustion i...,brand,"{'text': ['iqbar'], 'answer_start': [164]}"
1,gomacro macrobar vegan bar peanut butter choco...,flavor,"{'text': ['peanut butter chocolate chip'], 'an..."
2,gomacro macrobar vegan bar peanut butter choco...,kosher,{'text': ['are also organic gluten-free kosher...
3,flavor name: peanut_butternbsp; |nbsp; size:4 ...,taste?,"{'text': ['peanut butter'], 'answer_start': [6..."
4,stay focused active throughout the day with i...,fats grams?,"{'text': ['15g of fats'], 'answer_start': [2505]}"


In [ ]:
qa_test_dataset = Dataset.from_pandas(dataset_df)
datasets = qa_test_dataset

In [ ]:
custom_datasets = DatasetDict({'train': datasets, 'validation': datasets})

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [ ]:
squad_datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
custom_datasets

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', '__index_level_0__'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', '__index_level_0__'],
        num_rows: 300
    })
})

In [ ]:
squad_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
custom_datasets["train"][0]
datasets = custom_datasets

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"], 2)

,context,question,answers,__index_level_0__
0,powercrunch protein energy bar - peanut butter creme high protein creme filled wafer bar 13 grams of protein 5 grams of sugar proto whey for maximum protein absorption proto whey is a revolutionary new form of dietary protein. its micro peptidestm are designed exactly to specifications of human metabolism (dieting muscle repair growth energy immune response). ** **these statements have not been evaluated by the food and drug administration. this product is not intended to diagnose treat cure or prevent disease. 13 grams protein 05 grams sugar the einstein of protein,how many g of sugar?,"{'answer_start': [109], 'text': ['5 grams of sugar']}",266
1,pure protein lemon cake bars provide high-quality protein for a convenient pre or post workout snack. with 20 grams of protein 3 grams of sugar and 200 calories now you can have your cake and eat it to! take your pure protein lemon cake bar with you anywhere for a sensationally nutritious snack. pure protein bars lemon cake 20 g protein 1.76 oz 6 ct: one 6 count box of pure protein lemon cake protein bars gluten free protein bars with low sugar * made with non-gmo ingredients and whey protein isolate low sugar * protein bars have 2g sugar per bar 20g protein bars support lean muscle and strength as well as an active lifestyle enjoy protein snack bars pre workout post workout or on the go *pure protein products contain 5g of sugar or less.,how many per box?,"{'answer_start': [357], 'text': ['6 count box']}",192


## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [ ]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors


Without any truncation, we get the following length for the input IDs:

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

657

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several: 

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 384, 153]

And if we decode them, we can see the overlap:

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] brand [SEP] we live in a world where physical exhaustion is typical and mental fatigue is inevitable. getting through your busy days depends on what you use to fuel your body. iqbar makes functional snacks and drinks that improve mind and body performance. say goodbye to sugary drinks carb - heavy snacks and empty calories that leave you with brain fog and low energy. iqbar and iqmix deliver products to support both your brain and body. iqbars are clean - label plant - based low - carb protein bars that keep you full with good fats and fibers and support mental clarity longevity with 6 brain - focused nutrients in each bar. iqmix hydration mix packets hydrate with electrolytes while supporting mental clarity and mood. keto electrolytes powder packets for the brain amp ; body : iqmix electrolyte powder packets do more than just help replenish lost sodium and potassium. our electrolyte mix is designed to rapidly rehydrate your body while keeping you energized focused amp ; and thin

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 5), (0, 0), (0, 2), (3, 7), (8, 10), (11, 12), (13, 18), (19, 24), (25, 33), (34, 44), (45, 47), (48, 55), (56, 59), (60, 66), (67, 74), (75, 77), (78, 88), (88, 89), (90, 97), (98, 105), (106, 110), (111, 115), (116, 120), (121, 128), (129, 131), (132, 136), (137, 140), (141, 144), (145, 147), (148, 152), (153, 157), (158, 162), (162, 163), (164, 166), (166, 169), (170, 175), (176, 186), (187, 193), (194, 197), (198, 204), (205, 209), (210, 217), (218, 222), (223, 226), (227, 231), (232, 243), (243, 244), (245, 248), (249, 256), (257, 259), (260, 265), (265, 266), (267, 273), (274, 277), (277, 278), (278, 279), (279, 284), (285, 291), (292, 295), (296, 301), (302, 305), (305, 310), (311, 315), (316, 321), (322, 325), (326, 330), (331, 336), (337, 340), (341, 344), (345, 348), (349, 355), (355, 356), (357, 359), (359, 362), (363, 366), (367, 369), (369, 371), (371, 372), (373, 380), (381, 389), (390, 392), (393, 400), (401, 405), (406, 410), (411, 416), (417, 420), (421, 4

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

brand brand


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

34 35


And we can double check that it is indeed the theoretical answer:

In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

iqbar
iqbar


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
features = prepare_train_features(datasets['train'][:2])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-custom",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-squad"` or `"huggingface/bert-finetuned-squad"`).

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

KeyboardInterrupt: ignored

We can now finetune our model by just calling the `train` method:

In [ ]:
import torch
print(torch.cuda.memory_allocated())

1337197056


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 368
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 1104


Epoch,Training Loss,Validation Loss
1,No log,1.112759
2,2.162200,0.849360
3,1.268800,0.780794


***** Running Evaluation *****
  Num examples = 368
  Batch size = 1
Saving model checkpoint to distilbert-base-uncased-finetuned-custom/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned-custom/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned-custom/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-custom/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-custom/checkpoint-500/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-custom/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-custom/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 368
  Batch size = 1
Saving model checkpoint to distilbert-base-uncased-finetuned-custom/checkpoint-1000
Configuration saved in distilbert-base-uncased-finetuned-custom/checkpoint-1000/config.json
Model wei

TrainOutput(global_step=1104, training_loss=1.629091159157131, metrics={'train_runtime': 1109.3842, 'train_samples_per_second': 0.995, 'train_steps_per_second': 0.995, 'total_flos': 768969389481984.0, 'train_loss': 1.629091159157131, 'epoch': 3.0})

Since this training is particularly long, let's save the model just in case we need to restart.

In [ ]:
trainer.save_model("qanda-bert-squad-trained-finetuned-custom")

Saving model checkpoint to qanda-bert-squad-trained-finetuned-custom
Configuration saved in qanda-bert-squad-trained-finetuned-custom/config.json
Model weights saved in qanda-bert-squad-trained-finetuned-custom/pytorch_model.bin
tokenizer config file saved in qanda-bert-squad-trained-finetuned-custom/tokenizer_config.json
Special tokens file saved in qanda-bert-squad-trained-finetuned-custom/special_tokens_map.json
Saving model checkpoint to distilbert-base-uncased-finetuned-custom
Configuration saved in distilbert-base-uncased-finetuned-custom/config.json
Model weights saved in distilbert-base-uncased-finetuned-custom/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-custom/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-custom/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.36k/1.24G [00:00<?, ?B/s]

Upload file runs/Mar08_07-56-59_0996c7eabc3d/events.out.tfevents.1646726308.0996c7eabc3d.77.0:  71%|#######1  …

To https://huggingface.co/kamilali/distilbert-base-uncased-finetuned-custom
   086f8e2..85b705e  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/kamilali/distilbert-base-uncased-finetuned-custom
   85b705e..eecdf36  main -> main



In [ ]:
trainer.push_to_hub("trained model")

Saving model checkpoint to distilbert-base-uncased-finetuned-custom
Configuration saved in distilbert-base-uncased-finetuned-custom/config.json
Model weights saved in distilbert-base-uncased-finetuned-custom/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-custom/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-custom/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}


## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [ ]:
from transformers import AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [ ]:
custom_tokenizer = AutoTokenizer.from_pretrained("kamilali/distilbert-base-uncased-finetuned-custom")
custom_model = AutoModelForQuestionAnswering.from_pretrained("kamilali/distilbert-base-uncased-finetuned-custom")

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

In [ ]:
# print("Provide a description of the product:", '\n')
# The Kind Bar has 15 grams of sugar and 10 grams of protein.
# text = input()

# print("\nProvide questions to ask the model, and press enter twice when finished:", '\n')
# questions = [
#         "What is the bar?",
#         "How many grams of sugar does it have?",
#         "How many grams of protein does it have?"
# ]

# cq = input()
# questions = []
# while cq != "":
#     questions.append(cq)
#     cq = input()

# print("Context provided:", text)
# print("Questions asked:", questions)

# print("-----")
import torch
from tqdm.auto import tqdm

predictions = []
custom_predictions = []
references = []

model = model.to('cuda')
custom_model = custom_model.to('cuda')
test = 0
for idx, entry in enumerate(tqdm(datasets['validation'])):
  context = entry['context']
  question = entry['question']
  answers = entry['answers']
  
  inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=512, return_tensors="pt")
  inputs = inputs.to('cuda')
  input_ids = inputs["input_ids"].tolist()[0]
  text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  base_out = model(**inputs)
  answer_start_scores, answer_end_scores = base_out[0], base_out[1]
  answer_start = torch.argmax(answer_start_scores)
  answer_end = torch.argmax(answer_end_scores) + 1

  inputs = custom_tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=512, return_tensors="pt")
  inputs = inputs.to('cuda')
  input_ids = inputs["input_ids"].tolist()[0]
  text_tokens = custom_tokenizer.convert_ids_to_tokens(input_ids)
  custom_out = custom_model(**inputs)
  custom_answer_start_scores, custom_answer_end_scores = custom_out[0], custom_out[1]
  custom_answer_start = torch.argmax(custom_answer_start_scores)
  custom_answer_end = torch.argmax(custom_answer_end_scores) + 1
  
  if custom_answer_start < custom_answer_end and answer_start < answer_end:
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    custom_answer = custom_tokenizer.convert_tokens_to_string(custom_tokenizer.convert_ids_to_tokens(input_ids[custom_answer_start:custom_answer_end]))
    if 'brand' in question:
      print(answer)
      print(custom_answer)
      print(answers)
      print(question)
      print(context)
      if test == 20:
        break
      test+=1
    predictions.append({
        "id": str(idx),
        "prediction_text": answer
    })
    custom_predictions.append({
        "id": str(idx),
        "prediction_text": custom_answer
    })
    references.append({
        "id": str(idx),
        "answers": answers
    })
# for question in questions:
#     inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
#     input_ids = inputs["input_ids"].tolist()[0]
 
#     text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
#     out = model(**inputs)
#     answer_start_scores, answer_end_scores = out[0], out[1]
 
#     answer_start = torch.argmax(answer_start_scores)
#     answer_end = torch.argmax(answer_end_scores) + 1
 
#     answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
 
#     print(f"Question: {question}")
#     print(f"Answer: {answer}\n")

  0%|          | 0/300 [00:00<?, ?it/s]

[CLS] brand [SEP] we live in a world where physical exhaustion is typical and mental fatigue is inevitable. getting through your busy days depends on what you use to fuel your body. iqbar makes functional snacks and drinks that improve mind and body performance. say goodbye to sugary drinks carb - heavy snacks and empty calories that leave you with brain fog and low energy. iqbar and iqmix deliver products to support both your brain and body. iqbars are clean - label plant - based low - carb protein bars that keep you full with good fats and fibers and support mental clarity longevity with 6 brain - focused nutrients in each bar. iqmix hydration mix packets hydrate with electrolytes while supporting mental clarity and mood. keto electrolytes powder packets for the brain amp ; body : iqmix electrolyte powder packets do more than just help replenish lost sodium and potassium. our electrolyte mix is designed to rapidly rehydrate your body while keeping you energized focused amp ; and thin

KeyboardInterrupt: ignored

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")
print(metric.compute(predictions=predictions, references=references))
print(metric.compute(predictions=custom_predictions, references=references))

{'exact_match': 11.578947368421053, 'f1': 37.88331390508782}
{'exact_match': 57.19298245614035, 'f1': 75.07970438097121}


In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

from transformers import default_data_collator

data_collator = default_data_collator

args = TrainingArguments(
    "eval",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

custom_trainer = Trainer(
    custom_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=custom_tokenizer,
)

In [ ]:
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
    custom_output = custom_trainer.model(**batch)
print(output.keys())
print(custom_output.keys())

odict_keys(['loss', 'start_logits', 'end_logits'])
odict_keys(['loss', 'start_logits', 'end_logits'])


The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([1, 384]), torch.Size([1, 384]))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([0], device='cuda:0'), tensor([0], device='cuda:0'))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

custom_start_logits = custom_output.start_logits[0].cpu().numpy()
custom_end_logits = custom_output.end_logits[0].cpu().numpy()

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

custom_start_indexes = np.argsort(custom_start_logits)[-1 : -n_best_size - 1 : -1].tolist()
custom_end_indexes = np.argsort(custom_end_logits)[-1 : -n_best_size - 1 : -1].tolist()

valid_answers = []
custom_valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )
for custom_start_index in custom_start_indexes:
    for custom_end_index in custom_end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            custom_valid_answers.append(
                {
                    "score": custom_start_logits[custom_start_index] + custom_end_logits[custom_end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
print(datasets["validation"].column_names)
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

['context', 'question', 'answers', '__index_level_0__']


  0%|          | 0/1 [00:00<?, ?ba/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [ ]:
raw_predictions = trainer.predict(validation_features)
custom_raw_predictions = custom_trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 368
  Batch size = 1


The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 368
  Batch size = 1


The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

custom_start_logits = custom_output.start_logits[0].cpu().numpy()
custom_end_logits = custom_output.end_logits[0].cpu().numpy()

offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

custom_start_indexes = np.argsort(custom_start_logits)[-1 : -n_best_size - 1 : -1].tolist()
custom_end_indexes = np.argsort(custom_end_logits)[-1 : -n_best_size - 1 : -1].tolist()

valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
print(valid_answers)


custom_valid_answers = []
for custom_start_index in custom_start_indexes:
    for custom_end_index in custom_end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            custom_start_index >= len(offset_mapping)
            or custom_end_index >= len(offset_mapping)
            or offset_mapping[custom_start_index] is None
            or offset_mapping[custom_end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if custom_end_index < custom_start_index or custom_end_index - custom_start_index + 1 > max_answer_length:
            continue
        if custom_start_index <= custom_end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[custom_start_index][0]
            end_char = offset_mapping[custom_end_index][1]
            custom_valid_answers.append(
                {
                    "score": custom_start_logits[custom_start_index] + custom_end_logits[custom_end_index],
                    "text": context[start_char: end_char]
                }
            )
custom_valid_answers = sorted(custom_valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
print(custom_valid_answers)

[{'score': -5.9942484, 'text': 'iqbar'}, {'score': -6.7989516, 'text': 'iqmix'}, {'score': -7.0175233, 'text': 'iqbar and iqmix'}, {'score': -7.459014, 'text': 'iqbar makes functional snacks and drinks that improve mind and body performance'}, {'score': -7.800829, 'text': 'iqbar makes functional snacks and drinks'}, {'score': -8.128316, 'text': 'iqbar makes functional snacks'}, {'score': -8.1322, 'text': 'iqbar'}, {'score': -8.63231, 'text': 'physical exhaustion is typical and mental fatigue is inevitable. getting through your busy days depends on what you use to fuel your body. iqbar'}, {'score': -8.660816, 'text': 'iqmix deliver products to support both your brain and body. iqbars'}, {'score': -8.879388, 'text': 'iqbar and iqmix deliver products to support both your brain and body. iqbars'}, {'score': -8.948674, 'text': 'iqmix deliver products to support both your brain and body. iqbars are clean-label plant-based low-carb protein bars'}, {'score': -8.981878, 'text': 'getting through

We can compare to the actual ground-truth answer:

In [ ]:
print(custom_raw_predictions[0])
datasets["validation"][0]["answers"]

(array([[  1.912769 ,  -7.73437  ,   1.9127568, ...,  -5.8637624,
         -8.683794 ,   1.9126955],
       [  2.093357 ,  -3.569177 ,   2.0933435, ...,  -8.225047 ,
         -8.533969 ,   2.0933   ],
       [ -1.8990293,  -6.9295173,  -1.8990363, ..., -10.190864 ,
        -10.21762  , -10.234484 ],
       ...,
       [ -5.8796554,  -5.2513375,  -6.708262 , ..., -10.125325 ,
        -10.102566 , -10.10406  ],
       [ -5.1295304,  -6.496382 ,  -7.2224436, ..., -10.460267 ,
        -10.463346 , -10.461811 ],
       [ -5.4983716,  -6.3115363,  -6.91836  , ..., -10.430057 ,
        -10.444246 , -10.453883 ]], dtype=float32), array([[ 2.3497887, -7.1511865,  2.349779 , ..., -7.607554 , -6.7042437,
         2.3496602],
       [ 2.4991047, -4.139352 ,  2.4990902, ..., -7.6599565, -5.465843 ,
         2.498998 ],
       [-1.1453446, -6.1138983, -1.1453482, ..., -9.60868  , -9.684755 ,
        -9.721168 ],
       ...,
       [-1.0183372, -6.212066 , -5.547434 , ..., -9.242025 , -9.235429 ,
   

{'answer_start': [164], 'text': ['iqbar']}

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

KeyError: ignored

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10784 features.


Then we can load the metric from the datasets library.

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 76.74550614947965, 'f1': 85.13412652023338}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("sgugger/my-awesome-model")
```

In [ ]:
print(valid_answers)

[{'score': -5.9942484, 'text': 'iqbar'}, {'score': -6.7989516, 'text': 'iqmix'}, {'score': -7.0175233, 'text': 'iqbar and iqmix'}, {'score': -7.459014, 'text': 'iqbar makes functional snacks and drinks that improve mind and body performance'}, {'score': -7.800829, 'text': 'iqbar makes functional snacks and drinks'}, {'score': -8.128316, 'text': 'iqbar makes functional snacks'}, {'score': -8.1322, 'text': 'iqbar'}, {'score': -8.63231, 'text': 'physical exhaustion is typical and mental fatigue is inevitable. getting through your busy days depends on what you use to fuel your body. iqbar'}, {'score': -8.660816, 'text': 'iqmix deliver products to support both your brain and body. iqbars'}, {'score': -8.879388, 'text': 'iqbar and iqmix deliver products to support both your brain and body. iqbars'}, {'score': -8.948674, 'text': 'iqmix deliver products to support both your brain and body. iqbars are clean-label plant-based low-carb protein bars'}, {'score': -8.981878, 'text': 'getting through